## Toy example: Inferring the mean of Gaussians

#### comparing the multi-round SNPE approach against our new incremental approach.

Goal of this little toy example is to show that provided our parameters are independent of each other, we need less simulations to derive a good approximation of our parameters.

In [40]:
import sys
sys.path.append('../code/')

import utils
from utils.helpers import get_time
from utils import inference

from utils.sbi_modulated_functions import Combined


from utils.helpers import get_time



# sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.inference import SNPE_C

import sbi

import pyknos

In [41]:
print(sbi.__version__)

0.17.2


In [42]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def Gaussian(thetas, normal_noise=0.1):
    
    gauss_list = []
    
    for theta in thetas:
    
        mu, sigma = theta, normal_noise # mean and standard deviation

        s = np.random.normal(mu, sigma, 1)
        
        gauss_list.append(s[0])
        
    gauss_obs = torch.tensor(gauss_list)
    
    return gauss_obs
    



### Calculate posterior for different number of simulations: 1k,  3k, 5k, 10k

### starting with multi-round snpe

In [47]:
true_thetas = torch.tensor([[3.0, 6.0, 20.0, 10.0, 90.0, 55.0, 27.0, 29.0, 4.0, 70.0, 5.0, 66.0, 99.0, 40.0, 45.0]], dtype=torch.float32)
parameter_names = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15']

prior_max = [100.0] * 15
prior_min = [0.1] * 15

print(prior_max)

import datetime

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [49]:
print(true_thetas)

tensor([[ 3.,  6., 20., 10., 90., 55., 27., 29.,  4., 70.,  5., 66., 99., 40.,
         45.]])


In [50]:
num_simulations_list = [500, 1000]


posterior_snpe_list = []

obs_real = Gaussian(true_thetas[0])

#obs_real = torch.tensor(obs_real, dtype=torch.float32)

print(obs_real)

for num_simulations in num_simulations_list:
    
    
    prior = utils.torchutils.BoxUniform(low=prior_min, high = prior_max)
    inf = SNPE(prior, density_estimator="mdn")
    simulator_stats, prior = prepare_for_sbi(Gaussian, prior)

    proposal = prior
    

    for i in range(3):
        
        start_time = datetime.datetime.now()

        theta, x = simulate_for_sbi(
            simulator_stats,
            proposal=proposal,
            num_simulations=num_simulations,
            num_workers=8,
        )

        inf = inf.append_simulations(theta, x, proposal=proposal)
        density_estimator = inf.train()

        posterior = inf.build_posterior(density_estimator)



        proposal = posterior.set_default_x(obs_real)
        
        finish_time = datetime.datetime.now()

        diff_time_snpe = finish_time - start_time
        
        print('for round ',i, ' time is: ', diff_time_snpe)

    posterior_snpe = posterior
    
    posterior_snpe_list.append(posterior_snpe)
    
finish_time = datetime.datetime.now()

diff_time_snpe = finish_time - start_time

tensor([ 3.0500,  5.9777, 19.9744,  9.9116, 90.0515, 55.3153, 27.0714, 29.0864,
         3.8423, 70.0605,  4.9874, 65.8916, 99.1237, 40.0247, 44.9965],
       dtype=torch.float64)


Running 500 simulations in 500 batches.:   0%|          | 0/500 [00:00<?, ?it/s]

0.5
 Neural network successfully converged after 56 epochs.for round  0  time is:  0:00:17.843562


Drawing 500 posterior samples:   0%|          | 0/500 [00:00<?, ?it/s]

Running 500 simulations in 500 batches.:   0%|          | 0/500 [00:00<?, ?it/s]

...

0.5
 Neural network successfully converged after 59 epochs.for round  0  time is:  0:00:36.418636


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Running 1000 simulations in 1000 batches.:   0%|          | 0/1000 [00:00<?, ?it/s]

Using SNPE-C with non-atomic loss
 Training neural network. Epochs trained: 21

AssertionError: NaN/Inf present in proposal posterior eval.

In [25]:
print(diff_time_snpe)

0:57:44.220780
